## <5-3 학습 마친 모델을 실전 투입하기>  

학습을 마친 문장 쌍 분류 모델을 인퍼런스하는 과정을 실습해 보겠습니다.  
이번 실습에서 만드는 웹 서비스의 개념도는 다음과 같습니다. 

<center><그림 5-4 문장 쌍 분류 웹 서비스></center>

<p align="center"><img src="https://i.imgur.com/I4lGm3J.jpg">  

<center>출처 : ratsgo's NLPBOOK</center>


전제와 가설 문장을 받아 답변하는 웹 서비스인데요, 전제와 가설 각각을 토큰화, 인덱싱한 뒤 모델 입력값으로 만들고 이를 모델에 넣어 [전제에 대해 가설이 참일 확률, 전제에 대해 가설이 거짓일 확률, 전제에 대해 가설이 중립일 확률]을 계산합니다.  
이후 약간의 후처리 과정을 거쳐 응답하는 방식입니다.

### 전제와 가설을 검증하는 자연어 추론 모델 만들기
---
<**1단계**> **코랩 노트북 초기화하기**   
이번 실습은 웹 브라우저에서 다음 주소(bit.ly/3iHQZea)에 접속하면 코랩 환경에서 수행할 수 있습니다. 이전 실습과 마찬가지로 코랩에서 `[내 드라이브에 복사]`를 진행하고 런타임 유형에서 하드웨어 가속을 사용하지 않도록 `[None]`로 선택합니다. 

<**2단계**> **환경 설정하기**   
다음 셀 명령으로 의존성 있는 패키지를 설치합니다.

In [1]:
#코드 5-16 의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 279 kB/s 
     |████████████████████████████████| 2.8 MB 9.6 MB/s 
     |████████████████████████████████| 806 kB 50.9 MB/s 
     |████████████████████████████████| 57 kB 3.5 MB/s 
     |████████████████████████████████| 829 kB 42.3 MB/s 
     |████████████████████████████████| 134 kB 45.2 MB/s 
     |████████████████████████████████| 398 kB 49.2 MB/s 
     |████████████████████████████████| 636 kB 39.4 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 67 kB 2.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.7 MB/s 
     |████████████████████████████████| 1.1 MB 44.1 MB/s 
     |████████████████████████████████| 96 kB 5.3 MB/s 
     |████████████████████████████████| 144 kB 60.2 MB/s 
     |████████████████████████████████| 271 kB 59.0 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [2]:
#코드 5-17 구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이어서 다음 코드를 실행하면 각종 인퍼런스 설정을 수행합니다. `pretrained_model_name`과 `max_seq_length`, `downstream_model_dir` 모두 앞 절에서 적용한 그대로 입력해야 합니다.

In [3]:
#코드 5-18 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-paircls/epoch=0-val_loss=0.84.ckpt


<**3단계**> **토크나이저 및 모델 불러오기**  
다음 코드를 차례로 실행해 토크나이저를 초기화하고 앞 절에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [4]:
#코드 5-19 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [5]:
#코드 5-20 체크포인트 로드
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

그리고 파인튜닝 때 사용한 `pretrained_model_name`에 해당하는 모델의 설정값들을 읽어들여 해당 값대로 BERT 모델을 초기화합니다.

In [6]:
#코드 5-21 BERT 설정 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [7]:
#코드 5-22 BERT 모델 초기화
model = BertForSequenceClassification(pretrained_model_config)

다음 코드는 초기화한 BERT 모델에 코드 5-20의 체크포인트를 주입합니다.

In [8]:
#코드 5-23 체크포인트 주입하기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

이어서 모델을 평가 모드로 전환합니다.  
이는 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 합니다.

In [9]:
#코드 5-24 평가 모드로 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

<**4단계**> **모델 출력값 만들고 후처리하기**  
다음 코드는 인퍼런스 과정을 정의한 함수입니다. 전제(`premise`)와 가설(`hypothesis`)을 입력받아 각각 토큰화, 인덱싱을 수행한 뒤 `input_ids`, `attention_mask`, `token_type_ids`를 만듭니다.  
이들 입력값을 파이토치 텐서 자료형으로 변환한 뒤 모델에 입력합니다.

In [10]:
#코드 5-25 인퍼런스 함수
def inference_fn(premise, hypothesis):
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][2].item(), 2)
        if torch.argmax(prob) == 0:
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓 (contradiction)"
        else:
            pred = "중립 (neutral)"
    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
    }

모델 출력값(`outputs.logits`)은 소프트맥스 함수 적용 이전의 로짓 형태인데요, 여기에 소프트맥스 함수를 써서 모델 출력을 확률 형태로 바꿉니다. 그리고 약간 후처리하여 예측 확률의 최댓값이 참 위치(`0`)일 경우 해당 문장이 '참(entailment)', 거짓 위치(`1`)일 경우 '거짓(contradiction)', 중립 위치(`2`)일 경우 '중립(neutral)'이 되도록 `pred`값을 만듭니다.

한편 `entailment_width`, `contradiction_width`, `neutral_width`는 웹 페이지에서 참, 거짓, 중립 막대 길이를 조정한느 정보이므로 크게 신경 쓰지 않아도 됩니다.

<**5단계**> **웹 서비스 시작하기**   
인퍼런스 함수 `inference_fn`을 가지고 다음 코드를 실행하면 웹 서비스를 띄울 수 있습니다.  
파이썬의 플라스크를 활용한 앱입니다.

In [11]:
!mkdir /root/.ngrok2 && echo "authtoken: {279c6rgTRIX2kOe4VIaoCJXy0u8_2fAuAWUJwnY1Q22wwoqra}" > /root/.ngrok2/ngrok.yml

In [ ]:
#코드 5-26 웹 서비스
from ratsnlp.nlpbook.paircls import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.paircls.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/http/client.py",

<center><코드 출력 예시></center>

<p align="center"><img src="https://i.imgur.com/nGL9nEj.png">  


웹 브라우저로 http://de4dc525be1c.ngrok.io 에 접속하면 그림5-5 같은 화면을 만날 수 있습니다. 단 실행할 때마다 이 주소가 변동하니 실제 접속할 때는 직접 코드10을 실행해 당시 출력된 주소로 접근해야 합니다.


### 맺음말  
---
지금까지 우리는 문장 쌍 분류 모델을 만드는 과정을 실습했습니다. 자연어 추론 데이터(KLUE-NLI)를 가지고 실습했으므로 이 말뭉치로 학습한 모델은 두 문장의 의미/논리적 관계(참, 거짓, 중립)를 판단할 수 있습니다. 물론 다른 데이터를 쓴다면 다른 태스크를 수행할 수 있습니다.  
문장 쌍 분류는 두 문서 사이의 유사도(similarity) 혹은 관련도(relevance)를 따지는 검색(retrieval) 모델로도 발전시킬 수 있어 그 활용도가 높습니다.

<center><그림 5-5 웹 서비스를 실행한 결과></center>

<p align="center"><img src="https://i.imgur.com/eLYbd3y.png">  

<center>출처 : ratsgo's NLPBOOK</center>


<p align="center"><img src="https://i.imgur.com/D2Kfvuk.png">  

<center>출처 : ratsgo's NLPBOOK</center>

<p align="center"><img src="https://i.imgur.com/r7ira5x.png">  

<center>출처 : ratsgo's NLPBOOK</center>
